In [30]:
import datetime
import time
import requests
import sqlite3
from faker import Faker
from datetime import datetime
import numpy as np

In [46]:
#Task 1. Реалізуйте декоратор type_check, який перевіряє відповідність типів аргументів функції заданим типам і викидає виняток, якщо типи не збігаються.
def type_check(func):
    def wrapper(*args, **kwargs):
        func_ = func.__annotations__
        for i, arg in enumerate(args):
            if func_ and func_.get(list(func_.keys())[i], None) and not isinstance(arg, func_[list(func_.keys())[i]]):
                return TypeError(f"Argument '{list(func_.keys())[i]}' has invalid type: {type(arg).__name__}") #or raise TypeError
        return func(*args, **kwargs)
    return wrapper

In [47]:
@type_check
def example_function(num: int, text: str, my_list: list, my_dict: dict):
    print(f"Received: {num}, {text}, {my_list}, {my_dict}")

num=23
text='45'
my_list=[23]
my_dict=(23, 24)

example_function(num, text, my_list, my_dict)

TypeError("Argument 'my_dict' has invalid type: tuple")

In [29]:
#Task 2. Реалізуйте декоратор delay, який затримує виконання функції на вказану кількість секунд. 
#Функція повинна на заданий час давати shutdown, також повинна бути перевірка на статус код та try/except для перевірки connection 
def delay(func):
    def wrapper(*args, **kwargs):
        try:
            response = func(*args, **kwargs)
            if response.status_code == 200:
                time.sleep(15)
                return response.json()
                print('All good')
            elif response.status_code == 404:
                print('Error connect')
            else:
                 print(f'Unkwown status code {response.status_code}')
        except Exception as E:
            print(f'Unexpected error occured {E}')
    return wrapper


@delay
def connect_func(url):
    return requests.get(url)

data = connect_func('https://api.monobank.ua/bank/currency')
if data:
    print(data)


[{'currencyCodeA': 840, 'currencyCodeB': 980, 'date': 1700721673, 'rateBuy': 36.05, 'rateCross': 0, 'rateSell': 37.2995}, {'currencyCodeA': 978, 'currencyCodeB': 980, 'date': 1700690473, 'rateBuy': 39.31, 'rateCross': 0, 'rateSell': 40.7498}, {'currencyCodeA': 978, 'currencyCodeB': 840, 'date': 1700721673, 'rateBuy': 1.085, 'rateCross': 0, 'rateSell': 1.096}, {'currencyCodeA': 826, 'currencyCodeB': 980, 'date': 1700751353, 'rateBuy': 0, 'rateCross': 46.8043, 'rateSell': 0}, {'currencyCodeA': 392, 'currencyCodeB': 980, 'date': 1700751343, 'rateBuy': 0, 'rateCross': 0.2522, 'rateSell': 0}, {'currencyCodeA': 756, 'currencyCodeB': 980, 'date': 1700751355, 'rateBuy': 0, 'rateCross': 42.2978, 'rateSell': 0}, {'currencyCodeA': 156, 'currencyCodeB': 980, 'date': 1700751317, 'rateBuy': 0, 'rateCross': 5.2169, 'rateSell': 0}, {'currencyCodeA': 784, 'currencyCodeB': 980, 'date': 1700751313, 'rateBuy': 0, 'rateCross': 10.1595, 'rateSell': 0}, {'currencyCodeA': 971, 'currencyCodeB': 980, 'date': 16

In [ ]:
#SQL - tasks
# Task 1. create table Person, with id: int, primary key, name: text, surname: text,
#date_of_birth date 
# Connect with BD 


# create a connection to the database
conn = sqlite3.connect('example.db')

print(conn)
# create a table
conn.execute('''CREATE TABLE IF NOT EXISTS
             persons (id INTEGER PRIMARY KEY, name TEXT, surname TEXT, date_of_birth DATE)''')

# commit the changes
conn.commit()

# close the connection
conn.close()

In [ ]:
# Task 2. Заповнити таблицю Person 15 кортежі, *use - faker, *use - context manager


conn = sqlite3.connect('example.db')
fake = Faker()
Faker.seed(42)

with sqlite3.connect('example.db') as conn:
    conn.execute('''CREATE TABLE IF NOT EXISTS persons (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    surname TEXT,
                    date_of_birth DATE)
                ''')

    for i in range(1, 16):
        name = fake.first_name()
        surname = fake.last_name()
        date_of_birth = fake.date_of_birth(maximum_age=65, minimum_age=20) #indicated max and min age in order to avoid huge difference in randomly generated time data
        conn.execute("INSERT INTO persons (id, name, surname, date_of_birth) VALUES (?, ?, ?, ?)",
                     (i, name, surname, date_of_birth))

    conn.commit()

In [31]:
# check on inserted data
with sqlite3.connect('example.db') as conn:
    
    cursor = conn.cursor()
    cursor.execute("SELECT  * FROM Persons")
    rows = cursor.fetchall()

    # print the data
    for row in rows:
        print(row)

(1, 'Danielle', 'Johnson', '1976-08-09')
(2, 'John', 'Taylor', '1964-11-12')
(3, 'Erica', 'Mcclain', '1963-10-22')
(4, 'Brittany', 'Johnson', '1964-04-07')
(5, 'Jeffery', 'Wagner', '1959-09-12')
(6, 'Anna', 'Baldwin', '1994-12-22')
(7, 'Robert', 'Cole', '1976-10-27')
(8, 'Lisa', 'Smith', '1968-10-03')
(9, 'Helen', 'Peterson', '1968-06-21')
(10, 'Jason', 'Gallagher', '1964-11-05')
(11, 'Christopher', 'Davis', '1981-04-19')
(12, 'Cassandra', 'Roman', '1989-02-26')
(13, 'Amanda', 'Dudley', '1983-08-26')
(14, 'Christian', 'Carter', '2000-09-01')
(15, 'Christine', 'Hensley', '1997-03-07')


In [32]:
# Task 3. Знайти всіх людей у кого день народження 19-03-1998 або 12-10-1998
with sqlite3.connect('example.db') as conn:
    
    cursor = conn.cursor()
    cursor.execute("SELECT  * FROM Persons WHERE date_of_birth IN ('1998-03-19', '1998-10-12')" )
    rows = cursor.fetchall()

    if not rows:
        print ('There are no records matching your request in the table')
    else:
        for row in rows:
            print(row)

There are no records matching your request in the table


In [33]:
# Task 4. Знайте найстарших людей в наборі(топ 5)
with sqlite3.connect('example.db') as conn:
    
    cursor = conn.cursor()
    cursor.execute("SELECT  * FROM Persons ORDER BY date_of_birth LIMIT 5")
    rows = cursor.fetchall()

    # print the data
    for row in rows:
        print(row)

# or:
with sqlite3.connect('example.db') as conn:
    
    cursor = conn.cursor()
    cursor.execute("SELECT  * FROM Persons ORDER BY date_of_birth")
    rows = cursor.fetchmany(5)

    # print the data
    for row in rows:
        print(row)


(5, 'Jeffery', 'Wagner', '1959-09-12')
(3, 'Erica', 'Mcclain', '1963-10-22')
(4, 'Brittany', 'Johnson', '1964-04-07')
(10, 'Jason', 'Gallagher', '1964-11-05')
(2, 'John', 'Taylor', '1964-11-12')
(5, 'Jeffery', 'Wagner', '1959-09-12')
(3, 'Erica', 'Mcclain', '1963-10-22')
(4, 'Brittany', 'Johnson', '1964-04-07')
(10, 'Jason', 'Gallagher', '1964-11-05')
(2, 'John', 'Taylor', '1964-11-12')


In [34]:
# Task 5. Вивезти всіх у кого name більше surname 
with sqlite3.connect('example.db') as conn:
    
    cursor = conn.cursor()
    cursor.execute("SELECT  * FROM Persons WHERE LENGTH(name) > LENGTH(surname)")
    rows = cursor.fetchall()

    # print the data
    for row in rows:
        print(row)

(1, 'Danielle', 'Johnson', '1976-08-09')
(4, 'Brittany', 'Johnson', '1964-04-07')
(5, 'Jeffery', 'Wagner', '1959-09-12')
(7, 'Robert', 'Cole', '1976-10-27')
(11, 'Christopher', 'Davis', '1981-04-19')
(12, 'Cassandra', 'Roman', '1989-02-26')
(14, 'Christian', 'Carter', '2000-09-01')
(15, 'Christine', 'Hensley', '1997-03-07')


In [ ]:
# задачі Numpy

In [35]:
# Task 1. Create list with 50 elements, reshape in 3 dim
arr=np.arange(1, 51)
print(arr.reshape(5, 10), arr.reshape(50, 1), arr.reshape(25, 2))

[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]
 [31 32 33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48 49 50]] [[ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]
 [30]
 [31]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [38]
 [39]
 [40]
 [41]
 [42]
 [43]
 [44]
 [45]
 [46]
 [47]
 [48]
 [49]
 [50]] [[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]
 [17 18]
 [19 20]
 [21 22]
 [23 24]
 [25 26]
 [27 28]
 [29 30]
 [31 32]
 [33 34]
 [35 36]
 [37 38]
 [39 40]
 [41 42]
 [43 44]
 [45 46]
 [47 48]
 [49 50]]


In [36]:
# Task 2. Find elements, where elements more than 20 and less 30 
comp = (arr > 20) & (arr < 30)
print(comp, arr[comp], end = '\n\n')

[False False False False False False False False False False False False
 False False False False False False False False  True  True  True  True
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False] [21 22 23 24 25 26 27 28 29]



In [37]:
# Task 3. Find elements, where elements is even

print(arr[(arr % 2 == 0)]) 

[ 2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48
 50]


In [42]:
# Task 4.  Generate chess dashboard 

def chess():
    
    chessboard = np.zeros((8, 8), dtype=int)

    chessboard[1::2, ::2] = 1
    chessboard[::2, 1::2] = 1

    return chessboard

chessboard = chess()
print(chessboard)

[[0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]]


In [43]:
# Task 5. Find elements, where elements is NOT even, less than 41 and more then 15 and elements not equal 21

print(arr[(arr % 2 != 0) & (arr < 41) &(arr>15) & (arr !=21)]) 

[17 19 23 25]


In [45]:
#6 task # Task 6. Find sum of diagonal for matrix (5, 5). 
#Step 1: generate matrix. Step 2: display diagonal, Step 3: Find sum of main dianogal, 
#Step 3: Find alternative diagonal, find elements what more than sum of main diagonal
arr=(np.arange(1,26)).reshape(5,5)
main_dia=arr.diagonal()
alt_dia=np.flipud(arr).diagonal()  #(upside down dia option) OR alt_dia = np.fliplr(arr).diagonal()
print(f'Sum of the main diagonal {main_dia} is: {np.sum(main_dia)} \nSum of the alternative diagonal {alt_dia} is:{np.sum(alt_dia)} ')

Sum of the main diagonal [ 1  7 13 19 25] is: 65 
Sum of the alternative diagonal [21 17 13  9  5] is:65 
